# Cloud Workshop Azure Databricks
## 08. Cross Validation
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>
V1.4 29/06/2020

# Documentation
Présentation https://azure.microsoft.com/fr-fr/services/databricks/

Documentation Azure Databricks : https://docs.microsoft.com/fr-fr/azure/databricks/

Documentation Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/

Github : https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/azure-databricks

## Using Cross Validation

In this exercise, you will use cross-validation to optimize parameters for a regression model.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [4]:
import datetime
now = datetime.datetime.now()
print(now)

2020-07-06 08:18:53.781329

In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

# Load the source data
# File location and type
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

data = data.select("DayofMonth", "DayOfWeek", "Carrier", "OriginAirportID", "DestAirportID", "DepDelay", col("ArrDelay").alias("label"))

# Split the data
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]

### Define the Pipeline
Now define a pipeline that creates a feature vector and trains a regression model

In [7]:
monthdayIndexer = StringIndexer(inputCol="DayofMonth", outputCol="DayofMonthIdx")
weekdayIndexer = StringIndexer(inputCol="DayOfWeek", outputCol="DayOfWeekIdx")
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
originIndexer = StringIndexer(inputCol="OriginAirportID", outputCol="OriginAirportIdx")
destIndexer = StringIndexer(inputCol="DestAirportID", outputCol="DestAirportIdx")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normNums")

featVect = VectorAssembler(inputCols=["DayofMonthIdx", "DayOfWeekIdx", "CarrierIdx", "OriginAirportIdx", "DestAirportIdx", "normNums"], outputCol="features")

lr = LinearRegression(labelCol="label", featuresCol="features")

pipeline = Pipeline(stages=[monthdayIndexer, weekdayIndexer, carrierIndexer, originIndexer, destIndexer, numVect, minMax, featVect, lr])

### Tune Parameters
You can tune parameters to find the best model for your data. <br>
To do this you can use the  **CrossValidator** class to evaluate each combination of parameters defined in a **ParameterGrid** against multiple *folds* of the data split into training and validation datasets, in order to find the best performing parameters. <br> 
> Note that this can take a long time to run because every parameter combination is tried multiple times.

In [9]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.01]).addGrid(lr.maxIter, [10, 5]).build()
cv = CrossValidator(estimator=pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, numFolds=2)

model = cv.fit(train)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

### Test the Model
Now you're ready to apply the model to the test data.

In [11]:
prediction = model.transform(test)
predicted = prediction.select("features", "prediction", "label")

predicted.show()

+--------------------+--------------------+-----+
 features| prediction|label|
+--------------------+--------------------+-----+
[25.0,2.0,10.0,57...| 20.3948670160039| 41|
[25.0,2.0,10.0,49...| 93.1412421791783| 82|
[25.0,2.0,10.0,38...| 30.231702309718457| 27|
[25.0,2.0,10.0,38...| -2.3757427053878644| -19|
[25.0,2.0,10.0,48...| -3.1721430763666802| 28|
[25.0,2.0,10.0,12...| -1.3599304225842275| -20|
[25.0,2.0,10.0,12...| 44.45743086598101| 57|
[25.0,2.0,10.0,12...| 1.8844995477260653| 67|
[25.0,2.0,10.0,12...| 86.76631461239842| 82|
[25.0,2.0,10.0,12...| 2.808666049157573| 67|
[25.0,2.0,10.0,46...| -3.8842199892349143| -8|
[25.0,2.0,10.0,53...| 74.07447707532255| 62|
[25.0,2.0,10.0,0....| 58.952771073374564| 36|
[25.0,2.0,10.0,64...| -7.956222817689664| -25|
[25.0,2.0,2.0,11....| 17.194230286242778| 35|
[25.0,2.0,2.0,18....| 25.556209233692883| 49|
[25.0,2.0,2.0,23....|0.001520029278950119| 36|
[25.0,2.0,2.0,3.0...| 0.2300749834091178| -10|
[25.0,2.0,2.0,3.0...|-0.36099286597047353| -16|
[25.0,2.0,2.0,3.0...| -0.8640448580708835| -10|
+--------------------+--------------------+-----+
only showing top 20 rows

### Examine the Predicted and Actual Values
You can plot the predicted values against the actual values to see how accurately the model has predicted. In a perfect model, the resulting scatter plot should form a perfect diagonal line with each predicted value being identical to the actual value - in practice, some variance is to be expected.
Run the cells below to create a temporary table from the **predicted** DataFrame and then retrieve the predicted and actual label values using SQL. You can then display the results as a scatter plot, specifying **-** as the function to show the unaggregated values.

In [13]:
predicted.createOrReplaceTempView("regressionPredictions")

In [14]:
%sql
SELECT label, prediction FROM regressionPredictions

label,prediction
41,20.3948670160039
82,93.1412421791783
27,30.231702309718457
-19,-2.3757427053878644
28,-3.1721430763666802
-20,-1.3599304225842275
57,44.45743086598101
67,1.8844995477260653
82,86.76631461239842
67,2.808666049157573


### Retrieve the Root Mean Square Error (RMSE)
There are a number of metrics used to measure the variance between predicted and actual values. Of these, the root mean square error (RMSE) is a commonly used value that is measured in the same units as the prediced and actual values - so in this case, the RMSE indicates the average number of minutes between predicted and actual flight delay values. You can use the **RegressionEvaluator** class to retrieve the RMSE.

In [16]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 17.31014262829401